In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
pd.set_option('display.max_rows', 4)

In [10]:
df = pd.read_csv("Denver.csv")

In [11]:
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [12]:
df.rename(columns={'OFFENSE_TYPE_ID': 'Crime-type-old', 'OFFENSE_CATEGORY_ID': 'Crime-category',
                   'FIRST_OCCURRENCE_DATE': 'Crime-start-time', 'LAST_OCCURRENCE_DATE': 'Crime-end-time',
                   'REPORTED_DATE': 'Crime-report-time'}, inplace=True) # Renaming wrong names
df.drop(['INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE', 'OFFENSE_CODE_EXTENSION',
         'INCIDENT_ADDRESS', 'GEO_X', 'GEO_Y', 'GEO_LON', 'GEO_LAT', 'DISTRICT_ID',
         'PRECINCT_ID', 'NEIGHBORHOOD_ID', 'IS_CRIME', 'IS_TRAFFIC'], axis=1, inplace=True) # Dropping columns
df['Crime-key'] = df.index # Creates Date_Key column based on index
df["Crime-key"] += 193678 # Adds 193678 to every row in the column
cols = ["Crime-key", "Crime-type-old", "Crime-report-time", "Crime-start-time", 'Crime-end-time', 'Crime-category'] 
df = df[cols] # Reorganize the columns

In [13]:
# Cretes the new column mapped according to Vancouver dataset types
crime_types = df['Crime-type-old'].tolist()
type_mapping = {
    'Break and Enter Commercial': ['robbery-business', 'theft-from-bldg', 'robbery-bank', 'burglary-business-by-force'],
    'Break and Enter Residential/Other': ['stolen-property-buy-sell-rec', 'burglary-residence-by-force', 'property-crimes-other', 'burglary-residence-no-force', 'robbery-residence', 'theft-of-rental-property', 'burglary-safe'],
    'Homicide': ['threats-to-injure', 'homicide-family', 'traf-vehicular-homicide', 'homicide-conspiracy', 'extortion', 'homicide-other', 'police-false-information', 'homicide-police-by-gun', 'homicide-negligent'],
    'Mischief': ['fraud-nsf-closed-account', 'forgery-poss-of-forged-inst', 'drug-pcs-other-drug', 'escape', 'weapon-by-prev-offender-powpo', 'obscene-material-possess', 'police-interference', 'drug-cocaine-possess', 'criminal-mischief-other', 'obscene-material-mfr', 'forgery-poss-of-forged-ftd', 'violation-of-custody-order', 'impersonation-of-police', 'riot', 'explosives-posses', 'drug-hallucinogen-possess', 'weapon-carrying-concealed', 'arson-public-building', 'fraud-criminal-impersonation', 'drug-methamphetamine-mfr', 'forgery-other', 'drug-marijuana-cultivation', 'gambling-gaming-operation', 'drug-methampetamine-sell', 'liquor-misrepresent-age-minor', 'liquor-possession', 'pawn-broker-viol', 'forgery-posses-forge-device', 'weapon-carrying-prohibited', 'contraband-into-prison', 'money-laundering', 'forgery-checks', 'weapon-poss-illegal-dangerous', 'traffic-accident-dui-duid', 'drug-fraud-to-obtain', 'escape-aiding', 'public-peace-other', 'arson-vehicle', 'traf-habitual-offender', 'weapon-unlawful-sale', 'drug-forgery-to-obtain', 'littering', 'explosive-incendiary-dev-pos', 'drug-hallucinogen-mfr', 'drug-heroin-possess', 'gambling-betting-wagering', 'accessory-conspiracy-to-crime', 'drug-opium-or-deriv-sell', 'drug-synth-narcotic-possess', 'public-fighting', 'weapon-altering-serial-number', 'drug-make-sell-other-drug', 'drug-barbiturate-mfr', 'disturbing-the-peace', 'drug-barbiturate-sell', 'weapon-flourishing', 'fraud-by-use-of-computer', 'police-making-a-false-rpt', 'criminal-mischief-graffiti', 'eavesdropping', 'weapon-other-viol', 'riot-incite', 'criminal-trespassing', 'forgery-counterfeit-of-obj', 'wiretapping', 'weapon-unlawful-discharge-of', 'drug-synth-narcotic-sell', 'prostitution-aiding', 'fireworks-possession', 'drug-marijuana-sell', 'illegal-dumping', 'violation-of-court-order', 'disarming-a-peace-officer', 'liquor-manufacturing', 'vehicular-eluding-no-chase', 'sex-off-fail-to-register', 'drug-marijuana-possess', 'drug-cocaine-sell', 'contraband-possession', 'arson-business', 'drug-barbiturate-possess', 'bribery', 'sex-off-registration-viol', 'reckless-endangerment', 'probation-violation', 'drug-methampetamine-possess', 'drug-heroin-sell', 'police-resisting-arrest', 'police-obstruct-investigation', 'violation-of-restraining-order', 'failure-to-report-abuse', 'criminal-mischief-mtr-veh', 'arson-other', 'liquor-other-viol', 'fraud-by-telephone', 'altering-vin-number', 'weapon-fire-into-occ-veh', 'window-peeping', 'weapon-fire-into-occ-bldg', 'parole-violation', 'arson-residence', 'gambling-device', 'bomb-threat', 'failure-to-appear', 'drug-poss-paraphernalia', 'police-disobey-lawful-order', 'prostitution-engaging-in', 'indecent-exposure', 'animal-poss-of-dangerous', 'explosive-incendiary-dev-use', 'other-enviornment-animal-viol', 'loitering', 'drug-opium-or-deriv-possess', 'false-imprisonment', 'obstructing-govt-operation', 'liquor-sell', 'prostitution-pimping', 'vehicular-eluding', 'public-order-crimes-other', 'drug-hallucinogen-sell', 'animal-cruelty-to'],
    'Offence Against a Person': ['sex-aslt-w-object', 'sex-aslt-rape-pot', 'aggravated-assault-dv', 'sex-aslt-w-object-pot', 'sex-aslt-rape', 'curfew', 'harassment-stalking-dv', 'aslt-agg-police-gun', 'assault-police-simple', 'agg-aslt-police-weapon', 'harassment-sexual-in-nature', 'assault-dv', 'sex-aslt-non-rape-pot', 'aggravated-assault', 'harassment-dv', 'sex-aslt-fondle-adult-victim', 'menacing-felony-w-weap', 'sex-asslt-sodomy-man-strng-arm', 'kidnap-dv', 'sex-aslt-non-rape', 'kidnap-adult-victim', 'harassment', 'intimidation-of-a-witness'],
    'Other Theft': ['theft-confidence-game', 'theft-shoplift', 'theft-unauth-use-of-ftd', 'theft-embezzle', 'theft-other', 'theft-gas-drive-off', 'theft-stln-veh-const-eqpt', 'robbery-purse-snatch-w-force', 'theft-of-services', 'theft-purse-snatch-no-force', 'burglary-vending-machine', 'burglary-business-no-force', 'assault-simple', 'theft-from-yards', 'theft-of-cable-services', 'traf-other', 'robbery-street', 'burglary-poss-of-tools', 'theft-from-mails', 'theft-pick-pocket', 'fraud-identity-theft'],
    'Theft from Vehicle': ['burg-auto-theft-resd-no-force', 'burg-auto-theft-resd-w-force', 'burg-auto-theft-busn-w-force', 'theft-parts-from-vehicle', 'theft-items-from-vehicle', 'burg-auto-theft-busn-no-force', 'theft-stln-vehicle-trailer'],
    'Theft of Bicycle': ['theft-bicycle'],
    'Theft of Vehicle': ['robbery-car-jacking', 'theft-fail-return-rent-veh', 'theft-of-motor-vehicle', 'traf-vehicular-assault', 'traf-impound-vehicle'],
    'Vehicle Collision or Pedestrian Struck (with Fatality)': [],
    'Vehicle Collision or Pedestrian Struck (with Injury)': [],
    'Vehicle Collision or Pedestrian Struck (Fatal or not)': ['traffic-accident', 'traffic-accident-hit-and-run']
}

type_col = []
for c in crime_types:
    for k, v in type_mapping.items():
        if c in type_mapping[k]:
            type_col.append(k)
            break
            
df['Crime-type'] = type_col
df.drop(['Crime-type-old'], axis=1, inplace=True)

In [14]:
# Cretes the new column with the category of the crime-type
crime_types = df['Crime-type'].tolist()
category_mapping = {
    'Break and Enter Properties': ['Break and Enter Commercial', 'Break and Enter Residential/Other'],
    'Homicide': ['Homicide'],
    'Mischief': ['Mischief'],
    'Offence Against a Person': ['Offence Against a Person'],
    'Theft': ['Other Theft', 'Theft from Vehicle', 'Theft of Bicycle', 'Theft of Vehicle'],
    'Traffic': ['Vehicle Collision or Pedestrian Struck (with Fatality)', 'Vehicle Collision or Pedestrian Struck (with Injury)', 'Vehicle Collision or Pedestrian Struck (Fatal or not)'],
}

category_col = []
for c in crime_types:
    for k, v in category_mapping.items():
        if c in category_mapping[k]:
            category_col.append(k)
            break
            
df['Crime-category'] = category_col

In [15]:
# Creates the new column with the severity of each type of crime
crime_type_severity = []
for crime_type in df['Crime-type'].tolist():
    if crime_type == 'Homicide':
        crime_type_severity.append(8)
    elif crime_type == 'Offence Against a Person':
        crime_type_severity.append(7)
    elif crime_type == 'Vehicle Collision or Pedestrian Struck (with Fatality)':
        crime_type_severity.append(6)
    elif crime_type == 'Vehicle Collision or Pedestrian Struck (with Injury)' or \
        crime_type == 'Vehicle Collision or Pedestrian Struck (Fatal or not)':
        crime_type_severity.append(5)
    elif crime_type == 'Mischief':
        crime_type_severity.append(4)
    elif crime_type == 'Break and Enter Commercial' or \
    crime_type == 'Break and Enter Residential/Other':
        crime_type_severity.append(3)
    elif crime_type == 'Theft of Bicycle' or crime_type == 'Theft of Vehicle':
        crime_type_severity.append(2)
    elif crime_type == 'Other Theft' or crime_type == 'Theft from Vehicle':
        crime_type_severity.append(1) 
        
df['Crime-type-severity-index'] = crime_type_severity

In [16]:
# Convert string to datetime for all dates
start_time_datetime = []
end_time_datetime = []
report_time_datetime = []

for i in range(df.shape[0]):
    start_time_datetime.append(datetime.strptime(df.iloc[i]['Crime-start-time'], '%m/%d/%Y %I:%M:%S %p'))
    report_time_datetime.append(datetime.strptime(df.iloc[i]['Crime-report-time'], '%m/%d/%Y %I:%M:%S %p'))  
    

    if (isinstance(df.iloc[i]['Crime-end-time'], float)):
        end_time_datetime.append('NaN')
    else:
        end_time_datetime.append(datetime.strptime(df.iloc[i]['Crime-end-time'], '%m/%d/%Y %I:%M:%S %p'))



In [17]:
# Replace old columns by new columns
df.drop(['Crime-report-time', 'Crime-start-time', 'Crime-end-time'], axis=1, inplace=True)
df['Crime-report-time'] = report_time_datetime
df['Crime-start-time'] = start_time_datetime
df['Crime-end-time'] = end_time_datetime

In [18]:
# Organizes the columns
cols = ['Crime-key', 'Crime-report-time', 'Crime-start-time', 'Crime-end-time', 'Crime-type', 'Crime-category', 'Crime-type-severity-index']
df = df[cols]

In [19]:
df

,Crime-key,Crime-report-time,Crime-start-time,Crime-end-time,Crime-type,Crime-category,Crime-type-severity-index
0,193678,2016-06-15 23:31:00,2016-06-15 23:31:00,NaT,Mischief,Mischief,4
1,193679,2018-01-29 17:53:00,2017-10-11 12:30:00,2017-10-11 16:55:00,Other Theft,Theft,1
...,...,...,...,...,...,...,...
462403,656081,2019-11-17 16:52:00,2019-11-16 17:20:00,NaT,Mischief,Mischief,4
462404,656082,2019-12-27 07:57:00,2019-12-27 06:45:00,NaT,Vehicle Collision or Pedestrian Struck (Fatal ...,Traffic,5


In [20]:
df.to_csv("Denver_Crime_Final.csv", index=False) # Creates the csv file